# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [29]:
# imports
import os, requests
from IPython.display import Markdown
from dotenv import load_dotenv
from openai import OpenAI



In [21]:
# constants

MODEL_GPT = 'openai/gpt-5-nano'
MODEL_LLAMA = 'llama3.2:1b'

In [6]:
# set up environment
load_dotenv(override=True)

openai_api_key = os.getenv('OPENROUTER_API_KEY')
if openai_api_key:
    print(f"Openrouter API Key exists and begins {openai_api_key[:8]}")
else:
    print("Openrouter API Key not set")



Openrouter API Key exists and begins sk-or-v1


In [13]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": question}
    ]

In [30]:
# Get gpt-5-mini to answer, with streaming

openai = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openai_api_key,
)

response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))

- This code is inside a generator. It uses yield from to yield all values produced by the right-hand side.

- The right-hand side is a set comprehension: {book.get("author") for book in books if book.get("author")}
  - It iterates each dict in books and gets the value for the key "author" using get (which returns None if the key is missing).
  - The if clause filters out falsy values (None, "", 0, etc.), so only truthy authors are kept.
  - The set collects unique author values, removing duplicates (because sets cannot contain duplicates).

- yield from then yields each element of that set to the caller. Since sets are unordered, the authors are yielded in an unspecified order.

- Net effect: the generator yields each unique author that appears in books, ignoring entries without a truthy "author" value.

Notes and alternatives:
- If you need to preserve the order of first appearance, you could use a different approach, e.g.:
  - for author in (book.get("author") for book in books if book.get("author")):
      if author not in seen: yield author; seen.add(author)
  - or yield from dict.fromkeys(book.get("author") for book in books if book.get("author"))

- Example:
  - books = [{"author":"A"},{"author":"B"},{"author":"A"},{"author":""},{"author":None}]
  - The set comprehension yields {"A","B"}; yield from will yield "A" and "B" in some order.

In [ ]:
# Check if Ollama is running
requests.get("http://localhost:11434").content

b'Ollama is running'

In [27]:
# Get Llama 3.2 to answer

openai = OpenAI(
  base_url="http://localhost:11434/v1",
  api_key='ollama',
)

response = openai.chat.completions.create(model=MODEL_LLAMA, messages=messages)
result = response.choices[0].message.content
display(Markdown(result))

KeyboardInterrupt: 